In [1]:
import pandas as pd
import numpy as np
from datetime import date
import cpi
from xgboost import XGBRegressor

In [2]:
main_df = pd.read_csv(r'/home/shua/Desktop/msc-ml-datamining/Machine_Learning/project/v1/data/difference_df.csv').drop(['Unnamed: 0'], axis = 1)

/tmp/ipykernel_50928/446674581.py:1: DtypeWarning: Columns (3,4,12,15,146,245,246,250,260,276,288,289,290,291,294,295,296,297,298,300,301,302,303,304,305,309,311,312,313,314,319,320,321,322,323,324,325,326,327,331,332,333,335,336,337,339,342,344,345,351,352,354,356,357,358,360,361,363,364) have mixed types. Specify dtype option on import or set low_memory=False.
  main_df = pd.read_csv(r'/home/shua/Desktop/msc-ml-datamining/Machine_Learning/project/v1/data/difference_df.csv').drop(['Unnamed: 0'], axis = 1)


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
#expand the width of each column
pd.set_option('max_colwidth', 100)

#### Touching up the Target Variable

There are two potential target variables: 'sale_price', which was the price that the taggers downloaded from the web, and the 'sale.lastSale.price' which was the price gathered from the API.

Some quick EDA shows that both targets aren't entirely clean. To not trust entirely on the human taggers, we will focus on cleaning up the sale.lastSale.price and use this as our target variable

In [4]:
main_df.loc[:, ['sale_price', 'prev_sale_price', 'sale.lastSale.price', 'sale.priorSale.price']]

,sale_price,prev_sale_price,sale.lastSale.price,sale.priorSale.price
0,1030000.0,725000.0,1030000.0,725000.0
1,725000.0,NaN,1030000.0,725000.0
2,905000.0,680000.0,905000.0,680000.0
3,680000.0,NaN,905000.0,680000.0
4,1260000.0,844000.0,1260000.0,844000.0
...,...,...,...,...
4334,200000.0,NaN,355000.0,250000.0
4335,560000.0,335000.0,560000.0,335000.0
4336,335000.0,NaN,560000.0,335000.0
4337,200000.0,255000.0,255000.0,200000.0


In [5]:
df = main_df.dropna(subset = ['prev_sale_price']) #drop all rows without a target value

In [6]:
df.loc[:, ['sale_price', 'prev_sale_price', 'sale.lastSale.price', 'sale.priorSale.price']]

,sale_price,prev_sale_price,sale.lastSale.price,sale.priorSale.price
0,1030000.0,725000.0,1030000.0,725000.0
2,905000.0,680000.0,905000.0,680000.0
4,1260000.0,844000.0,1260000.0,844000.0
6,1317000.0,865000.0,1317000.0,865000.0
8,1205000.0,540000.0,1205000.0,850000.0
...,...,...,...,...
4329,335000.0,130000.0,335000.0,130000.0
4331,475000.0,359000.0,467000.0,359000.0
4333,354900.0,200000.0,355000.0,250000.0
4335,560000.0,335000.0,560000.0,335000.0


In [7]:
#fill sale.priorSale.price with prior_sale_price
df['sale.priorSale.price'] = df['sale.priorSale.price'].fillna(df['prev_sale_price'])
#fill sale.priorSale.saleDate with prev_sale_date
df['sale.priorSale.saleDate'] = df['sale.priorSale.saleDate'].fillna(df['prev_sale_year'])

/tmp/ipykernel_50928/3206577856.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sale.priorSale.price'] = df['sale.priorSale.price'].fillna(df['prev_sale_price'])
/tmp/ipykernel_50928/3206577856.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sale.priorSale.saleDate'] = df['sale.priorSale.saleDate'].fillna(df['prev_sale_year'])


In [8]:
#where sale.lastSale.price is equal to sale.priorSale.price, fill sale.priorSale.price with lower of sale_price and prev_sale_price
def fill_sale_nulls(row):
    if row['sale.lastSale.price'] == row['sale.priorSale.price']:
        row['sale.priorSale.price'] = min(row['sale_price'], row['prev_sale_price'])
        row['sale.priorSale.saleDate'] = min(row['sale_year'], row['prev_sale_year'])
    return row
df[df['sale.lastSale.price'] == df['sale.priorSale.price']] = df[df['sale.lastSale.price'] == df['sale.priorSale.price']].apply(fill_sale_nulls, axis = 1)


/tmp/ipykernel_50928/2568353309.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['sale.lastSale.price'] == df['sale.priorSale.price']] = df[df['sale.lastSale.price'] == df['sale.priorSale.price']].apply(fill_sale_nulls, axis = 1)


In [9]:
main_df = df.append(main_df.iloc[~df.index]).sort_index()

/tmp/ipykernel_50928/2558618823.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = df.append(main_df.iloc[~df.index]).sort_index()


In [317]:
address = ['address.houseNumber', 'street', 'city',
        'state', 'zip', 'address.zipPlus4', 'sold_month'
       ] #'address.latitude', 'address.longitude'

continuous = ['building.totalBuildingAreaSquareFeet',
       # 'building.unitCount',
       'building.bedroomCount', #'building.calculatedBathroomCount'
       'building.fullBathroomCount', 'building.bathroomCount',
       'lot.lotDepthFeet', 'lot.lotSizeSquareFeet','mls.lotSizeSquareFeet', 'mls.bedroomCount', 
       'building.storyCount', 'mls.floorCount', 'mls.bathroomCount',  'building.buildingQualityCode', 'assessment.totalAssessedValue','sale.priorSale.price',


 'bedrooms_tagged',
#  'bathrooms_tagged',
 'living_area_tagged',
#  'year_built_tagged',
#  'sub_type_tagged',
 'Bathroom_max_tagged',
 'Bathroom_min_tagged',
 'Bathroom_std_tagged',
 'Bathroom_mean_tagged',
 'Floor_max_tagged',
 'Floor_min_tagged',
 'Floor_std_tagged',
 'Floor_mean_tagged',
 'Walls_max_tagged',
 'Walls_min_tagged',
 'Walls_std_tagged',
 'Walls_mean_tagged',
 'Outview_tagged',
 'Outview_std_tagged',
#  's_tagged',
 'Kitchen_level_tagged',
 'Kitchen_level_std_tagged'] #'lot.lotSizeAcres', 'building.roomCount', 'lot.lotFrontageFeet', 'building.garageParkingSpaceCount',

    
#Notes: Need to reverse the ordinality of the buildingQualityCode

ordinal = ['general.propertyTypeDetail'] 


one_hot = ['building.styleCode', 'building.roofCover','building.exteriorWalls', 'building.heatSource'] #'mls.heatingTypes', 'mls.coolingTypes'
# 'building.garageCode'
  #'mls.coolingTypes','building.features' 'mls.exteriorConstruction', 'building.styleCode' 'mls.roofTypes','airConditioningSource',  
  #'airConditioningSourceCode': '1', 'interiorWallsCode', 'floorCoverCode', 'garageCode', 'heatSourceCode', 'patioCode' , 'porchCode', 'roofCoverCode, 'styleCode'

target = ['sale.lastSale.price']
target_year = ['sale.lastSale.saleDate']
# assessments = ['sale.priorSale.price', 'assessment.totalAssessedValue'] #Assessment imprvoed, assessment land
#Assessment year MUST BE the first entryy in the list
years = ['assessment.assessmentYear', target_year[0], 'sale.priorSale.saleDate', 'building.yearBuilt'] #BuildingYearBuilt  MUST BE the last entryy in the list
price_year_pairings = [('assessment.totalAssessedValue', 'assessment.assessmentYear'), ('sale.priorSale.price', 'sale.priorSale.saleDate'), (target[0], target_year[0])]

to_sparse = ['building.features', 'homeownerAssociations']

not_sure = [ 'building.heatSourceCode'] #OneHot? What to do with new values?

features_not_in_single = ['mls.totalBuildingAreaSquareFeet', 'mls.lotSizeSquareFeet', 'building.residentialUnitCount', 'building.buildingQualityCode', 'building.buildingQuality', 
'building.constructionTypeCode', 'building.constructionType', 'mls.daysOnMarket', 'mls.maxListPrice', 'mls.maxListPriceDate', 'mls.minListPrice', 'mls.minListPriceDate', 'mls.soldPrice', 'mls.soldDate', 'mls.bedroomCount', 'mls.bathroomCount', 'mls.floorCount',
'mls.propertyType', 'mls.appliances', 'sale.priorSale.price', 'sale.priorSale.saleDate']

In [318]:
main_df.columns

Index(['full_address', 'street', 'sale_year', 'prev_sale_year', 'sale_price',
       'prev_sale_price', 'price_difference', 'address', 'street_b', 'city',
       ...
       'Floor_min_tagged', 'Floor_std_tagged', 'Floor_mean_tagged',
       'Walls_max_tagged', 'Walls_min_tagged', 'Walls_std_tagged',
       'Walls_mean_tagged', 'Outview_tagged', 'Outview_std_tagged',
       's_tagged'],
      dtype='object', length=391)

In [319]:
features = [address, continuous, ordinal, years, target, one_hot, to_sparse, not_sure]
features = [x for y in features for x in y]
print( f' df rows: {main_df.shape[0]}, columns: {len(features)}')

 df rows: 4424, columns: 52


In [320]:
df = main_df.loc[:, features]
df

,address.houseNumber,street,city,state,zip,address.zipPlus4,sold_month,building.totalBuildingAreaSquareFeet,building.bedroomCount,building.fullBathroomCount,building.bathroomCount,lot.lotDepthFeet,lot.lotSizeSquareFeet,mls.lotSizeSquareFeet,mls.bedroomCount,building.storyCount,mls.floorCount,mls.bathroomCount,building.buildingQualityCode,assessment.totalAssessedValue,sale.priorSale.price,bedrooms_tagged,living_area_tagged,Bathroom_max_tagged,Bathroom_min_tagged,Bathroom_std_tagged,Bathroom_mean_tagged,Floor_max_tagged,Floor_min_tagged,Floor_std_tagged,Floor_mean_tagged,Walls_max_tagged,Walls_min_tagged,Walls_std_tagged,Walls_mean_tagged,Outview_tagged,Outview_std_tagged,Kitchen_level_tagged,Kitchen_level_std_tagged,general.propertyTypeDetail,assessment.assessmentYear,sale.lastSale.saleDate,sale.priorSale.saleDate,building.yearBuilt,sale.lastSale.price,building.styleCode,building.roofCover,building.exteriorWalls,building.heatSource,building.features,homeownerAssociations,building.heatSourceCode
0,3913,3913 york blvd,los angeles,ca,90065,3718,may,1121.0,2.0,2.0,2.0,800.0,4381.0,4381.0,2.0,1.0,1.0,2.0,1.0,747160,725000.0,2.0,1202.0,5.0,2.0,1.0,4.0,5.0,2.0,1.0,4.0,5.0,2.0,1.0,5.0,2.0,1.0,4.0,0.0,Single Family,2021,2021-04-19T00:00:00.000Z,2019-05-09T00:00:00.000Z,1939.0,1030000.0,15.0,Composition Shingle,Stucco,Central,"['Air Conditioning', 'Fireplace']","[{'type': 'PUD', 'feeAmount': 1200, 'feeFrequency': 'Annual'}]",3.0
1,3913,3913 york blvd,los angeles,ca,90065,3718,may,1121.0,2.0,2.0,2.0,800.0,4381.0,4381.0,2.0,1.0,1.0,2.0,1.0,747160,725000.0,2.0,1202.0,4.0,2.0,1.0,3.0,5.0,2.0,1.0,3.0,5.0,2.0,1.0,4.0,2.0,1.0,4.0,1.0,Single Family,2021,2021-04-19T00:00:00.000Z,2019-05-09T00:00:00.000Z,1939.0,1030000.0,15.0,Composition Shingle,Stucco,Central,"['Air Conditioning', 'Fireplace']","[{'type': 'PUD', 'feeAmount': 1200, 'feeFrequency': 'Annual'}]",3.0
2,4171,4171 division st,los angeles,ca,90065,4205,december,1250.0,2.0,1.0,1.0,890.0,5103.0,5103.0,2.0,1.0,1.0,2.0,7.0,905000,680000.0,2.0,1250.0,5.0,2.0,1.0,4.0,5.0,2.0,1.0,4.0,5.0,2.0,1.0,4.0,2.0,1.0,4.0,0.0,Single Family,2021,2020-11-23T00:00:00.000Z,2015-06-19T00:00:00.000Z,1947.0,905000.0,15.0,Composition Shingle,Stucco,Floor/Wall,[],[],6.0
3,4171,4171 division st,los angeles,ca,90065,4205,july,1250.0,2.0,1.0,1.0,890.0,5103.0,5103.0,2.0,1.0,1.0,2.0,7.0,905000,680000.0,2.0,1250.0,5.0,2.0,1.0,4.0,5.0,2.0,1.0,4.0,5.0,2.0,1.0,3.0,2.0,0.0,4.0,0.0,Single Family,2021,2020-11-23T00:00:00.000Z,2015-06-19T00:00:00.000Z,1947.0,905000.0,15.0,Composition Shingle,Stucco,Floor/Wall,[],[],6.0
4,4234,4234 palmero dr,los angeles,ca,90065,4251,september,1258.0,2.0,2.0,2.0,1100.0,8012.0,8012.0,2.0,1.0,1.0,2.0,9.0,1260000,844000.0,2.0,1258.0,5.0,2.0,1.0,4.0,5.0,2.0,1.0,3.0,5.0,2.0,1.0,4.0,2.0,1.0,4.0,1.0,Single Family,2021,2020-09-01T00:00:00.000Z,2017-02-09T00:00:00.000Z,1955.0,1260000.0,15.0,Composition Shingle,Stucco,Central,['Fireplace'],[],3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4334,2013,2013 n corlett ave,los angeles,ca,90059,3626,december,1796.0,4.0,2.0,2.0,1000.0,5003.0,4792.0,2.0,1.0,1.0,1.0,1.0,559254,250000.0,2.0,832.0,5.0,1.0,1.0,4.0,3.0,2.0,1.0,3.0,3.0,2.0,1.0,3.0,2.0,1.0,4.0,0.0,Single Family,2021,2017-04-18T00:00:00.000Z,2016-12-19T00:00:00.000Z,1945.0,355000.0,NaN,NaN,NaN,Floor/Wall,[],"[{'type': 'HOA', 'feeAmount': 400, 'feeFrequency': 'Monthly'}]",6.0
4335,2307,2307 e 120th st,los angeles,ca,90059,2629,august,753.0,2.0,1.0,1.0,2100.0,12186.0,12197.0,2.0,1.0,1.0,1.0,8.0,359185,335000.0,2.0,753.0,4.0,4.0,0.0,4.0,3.0,2.0,1.0,2.0,3.0,2.0,1.0,2.0,3.0,1.0,4.0,0.0,Single Family,2021,2021-07-26T00:00:00.000Z,2016-08-25T00:00:00.000Z,1915.0,560000.0,NaN,NaN,NaN,NaN,[],"[{'type': 'HOA', 'feeAmount': 300, 'feeFrequency': 'Annual'}]",NaN
4336,2307,2307 e 120th st,los angeles,ca,90059,2629,november,753.0,2.0,1.0,1.0,2100.0,121

In [321]:
df.dropna(subset = target, inplace= True) #drop all rows without a target value
df.replace('nan', np.nan, inplace = True) #replace all 'nan' strings with np.nan
df.shape[0]

4389

# Impute/Pre-Process

In [322]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4389 entries, 0 to 4338
Data columns (total 52 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   address.houseNumber                   4389 non-null   int64  
 1   street                                4389 non-null   object 
 2   city                                  4389 non-null   object 
 3   state                                 4389 non-null   object 
 4   zip                                   4389 non-null   object 
 5   address.zipPlus4                      4389 non-null   int64  
 6   sold_month                            4389 non-null   object 
 7   building.totalBuildingAreaSquareFeet  4385 non-null   float64
 8   building.bedroomCount                 4385 non-null   float64
 9   building.fullBathroomCount            4387 non-null   float64
 10  building.bathroomCount                4387 non-null   float64
 11  lot.lotDepthFeet 

In [323]:
# for col in ['sale_year', 'prev_sale_year']:  
#     year = np.array(df[col])
#     for i, x in enumerate(year):
#         try:
#             int(x)
#         except:
#             if pd.isnull(x) == False:
#                 try:
#                     year[i] = int(x.replace('-', ''))
#                 except:
#                     year[i] = int(float(x))

#     df[col] = year

#### Fix "Year" Features

In [324]:
def take_year(x):
    try:
        x = x.split('-')
        x = int(x[0])
    except:
        x = int(float(x))
    return x

current_year = int(date.today().year)
for year in years:
    print(year)
    
    #if year is an integer, fillna with mode. If year is a float, convert to int while handling the nan values.
    if df[year].dtype == 'int64':
        print('int64')
        df[year].fillna(df[year].mode()[0], inplace = True)
    elif df[year].dtype == 'float64':
        print('float64')
        df[year].fillna(df[year].mode()[0], inplace = True)
        df[year] = df.loc[:, year].astype(int)
        # df[df[year] > current_year] = current_year
    elif df[year].dtype == 'object':
        try:
            print('object')
            df[year].fillna(df[year].mode()[0], inplace=True)
            df[year] = df.loc[:, year].astype(int)

        except:
            print('else')
            df[year].fillna(df[year].mode()[0], inplace=True) #If not numerical, than apply the take_year function
            df[year] = df[year].apply(take_year)
            df.loc[:, year] = df.loc[:, year].astype(int)
    
    #drop row if length of the year string is not 4
    for x in df[year]:
        if len(str(x))<4 or len(str(x))>4:
            df.drop(df[df[year] == x].index, inplace = True)
        # df[df[year] > current_year] = current_year


assessment.assessmentYear
int64
sale.lastSale.saleDate
object
else
sale.priorSale.saleDate
object
else
building.yearBuilt
float64


#### Adjust Prices for Inflation

In [325]:
#Apply current price adjusted for inflation
from datetime import datetime
def apply_cpi(x, price_col, year_col):
    if pd.isnull(x.loc[price_col]) == False or x.loc != None:
        price = x.loc[price_col]
        year = x.loc[year_col]
        month_names = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
        month = x.loc['sold_month'][:3].lower()
        if month not in month_names:
            month = 'jan'
        if year == 2022 and month =='dec':
            month = 'nov'
        #convert month to its numerical value
        date_string = f"{year} {month}"
        date = datetime.strptime(date_string, "%Y %b")

        # if year != current_year:
            # print(price, year)
            #combine year and month to datetime format to get the correct CPI value
        # year = str(year) + '-' + str(month)
        val = cpi.inflate(price, date, items= 'Housing', )
        x.loc[price_col] = val
        return x
# df = df.apply(apply_cpi, args = ('sale.lastSale.price', 'sale.lastSale.saleDate'), axis = 1)
# df = df.apply(apply_cpi, args = ('assessment.totalAssessedValue', 'assessment.assessmentYear'), axis = 1)
for pair in price_year_pairings:
    print(pair)
    df = df.apply(apply_cpi, args = (pair), axis =1)

('assessment.totalAssessedValue', 'assessment.assessmentYear')
('sale.priorSale.price', 'sale.priorSale.saleDate')
('sale.lastSale.price', 'sale.lastSale.saleDate')


In [326]:
# c = df.select_dtypes(object).columns
# df[c] = df[c].apply(pd.to_numeric,errors='coerce')

In [327]:
df.columns

Index(['address.houseNumber', 'street', 'city', 'state', 'zip',
       'address.zipPlus4', 'sold_month',
       'building.totalBuildingAreaSquareFeet', 'building.bedroomCount',
       'building.fullBathroomCount', 'building.bathroomCount',
       'lot.lotDepthFeet', 'lot.lotSizeSquareFeet', 'mls.lotSizeSquareFeet',
       'mls.bedroomCount', 'building.storyCount', 'mls.floorCount',
       'mls.bathroomCount', 'building.buildingQualityCode',
       'assessment.totalAssessedValue', 'sale.priorSale.price',
       'bedrooms_tagged', 'living_area_tagged', 'Bathroom_max_tagged',
       'Bathroom_min_tagged', 'Bathroom_std_tagged', 'Bathroom_mean_tagged',
       'Floor_max_tagged', 'Floor_min_tagged', 'Floor_std_tagged',
       'Floor_mean_tagged', 'Walls_max_tagged', 'Walls_min_tagged',
       'Walls_std_tagged', 'Walls_mean_tagged', 'Outview_tagged',
       'Outview_std_tagged', 'Kitchen_level_tagged',
       'Kitchen_level_std_tagged', 'general.propertyTypeDetail',
       'assessment.asses

### Sparse House Features

In [328]:
to_sparse

['building.features', 'homeownerAssociations']

In [329]:
df.shape

(4389, 52)

In [330]:
houses_features = []
a = df.loc[:,['building.features']].values
for x in a:
    features_dict = {'AC': 0, 'Fireplace': 0, 'SolarHeat': 0, 'EvaporativeCooler': 0, 'AttachedGarage': 0, 'SpaorHotTub': 0, 'Spa': 0, 'Pool': 0, 'Garage': 0}
    for y in x:
        attr = y.replace("[", "").replace("]", "").replace("'", "").replace(" ", "").split(",")
        for item in attr:
            if "Conditioning" in item:
                if "Wall" in item:
                    features_dict['AC'] = 1
                elif "Central" in item:
                    features_dict['AC'] = 3
                else:
                    features_dict['AC'] = 2
            else:
                features_dict[item] = 1
    houses_features.append(features_dict)
features_df = pd.DataFrame(houses_features)
df = df.join(features_df).drop(['building.features'], axis = 1)

In [331]:
df.shape

(4389, 65)

### Sparse Homeowner Association

In [333]:
#grab the "type" and "amount" from the homeownerAssociations column. The column contains a dictionary with those two values

def get_hoa_type(x):
    if pd.isnull(x) == False:
        x = x.replace("[", "").replace("]", "").replace("'", "").replace(" ", "").split(",")
        for item in x:
            if 'type' in item:
                type = item.split(":")[1].replace("'", "").replace("}", "")
                return type
    else:
        return np.nan

def get_hoa_amount(x):
    if pd.isnull(x) == False:
        x = x.replace("[", "").replace("]", "").replace("'", "").replace(" ", "").split(",")
        for item in x:
            if 'feeAmount' in item:
                amount = int(item.split(":")[1].replace("'", "").replace("}", ""))
                return amount

    else:
        return np.nan

df['hoa_type'] = df.homeownerAssociations.apply(get_hoa_type)
df['hoa_amount'] = df.homeownerAssociations.apply(get_hoa_amount)
df.drop('homeownerAssociations', axis = 1, inplace = True)

Row Wise Fill-values (not overfit to the test set)

In [334]:
df.loc[:, 'mls.floorCount'].fillna(df.loc[:, 'building.storyCount'], inplace = True)
df.drop(['building.storyCount'], axis = 1, inplace = True)

df.loc[:, 'mls.bathroomCount'].fillna(df.loc[:, 'building.bathroomCount'], inplace = True)
df.drop(['building.bathroomCount'], axis = 1, inplace = True)

df.loc[:, 'mls.bedroomCount'].fillna(df.loc[:, 'building.bedroomCount'], inplace = True)
df.drop(['building.bedroomCount'], axis = 1, inplace = True)

df.loc[:, 'mls.lotSizeSquareFeet'].fillna(df.loc[:, 'lot.lotSizeSquareFeet'], inplace = True)
df.drop(['lot.lotSizeSquareFeet'], axis = 1, inplace = True)

df.loc[:, 'living_area_tagged'].fillna(df.loc[:, 'building.totalBuildingAreaSquareFeet'], inplace = True)
df.drop(['building.totalBuildingAreaSquareFeet'], axis = 1, inplace = True)
df.drop('', axis = 1, inplace = True)

In [335]:
df.columns

Index(['address.houseNumber', 'street', 'city', 'state', 'zip',
       'address.zipPlus4', 'sold_month', 'building.fullBathroomCount',
       'lot.lotDepthFeet', 'mls.lotSizeSquareFeet', 'mls.bedroomCount',
       'mls.floorCount', 'mls.bathroomCount', 'building.buildingQualityCode',
       'assessment.totalAssessedValue', 'sale.priorSale.price',
       'bedrooms_tagged', 'living_area_tagged', 'Bathroom_max_tagged',
       'Bathroom_min_tagged', 'Bathroom_std_tagged', 'Bathroom_mean_tagged',
       'Floor_max_tagged', 'Floor_min_tagged', 'Floor_std_tagged',
       'Floor_mean_tagged', 'Walls_max_tagged', 'Walls_min_tagged',
       'Walls_std_tagged', 'Walls_mean_tagged', 'Outview_tagged',
       'Outview_std_tagged', 'Kitchen_level_tagged',
       'Kitchen_level_std_tagged', 'general.propertyTypeDetail',
       'assessment.assessmentYear', 'sale.lastSale.saleDate',
       'sale.priorSale.saleDate', 'building.yearBuilt', 'sale.lastSale.price',
       'building.styleCode', 'building.roof

In [336]:
# drop porch, solarheadpool, undergroun basementgarage, and detached garage columns because they are pretty much empty

df.drop(['Porch', 'SolarHeatedPool', 'Underground/BasementGarage', 'DetachedGarage'], axis = 1, inplace = True )

In [337]:
# fill in missing values with the mode of the building exterior walls
df.loc[:, 'building.exteriorWalls'].fillna(df.loc[:, 'building.exteriorWalls'].mode()[0], inplace = True)

In [338]:
#Fill Kitchen_level_tagged with average value
df.loc[:, 'Kitchen_level_tagged'].fillna(df.loc[:, ['Bathroom_mean_tagged', 'Floor_mean_tagged', 'Walls_mean_tagged']].mean(axis = 1), inplace = True)
df.loc[:, 'Kitchen_level_std_tagged'].fillna(df.loc[:, ['Bathroom_std_tagged', 'Floor_std_tagged', 'Walls_std_tagged']].mean(axis = 1), inplace = True)

In [339]:
missing = df.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing = missing/df.shape[0]
missing

building.fullBathroomCount    0.000456
mls.bathroomCount             0.000456
mls.lotSizeSquareFeet         0.000911
mls.bedroomCount              0.000911
sale.priorSale.price          0.004329
                                ...   
hoa_type                      0.235589
building.styleCode            0.265892
hoa_amount                    0.308043
lot.lotDepthFeet              0.336523
building.roofCover            0.425382
Length: 14, dtype: float64

#### Going to Fill the rest of the fill values at the end of the data engineering notebook

In [259]:
df.to_csv('/home/shua/Desktop/msc-ml-datamining/Machine_Learning/project/v1/data/cleansed_df.csv')